# 数据载入、存储及文件格式

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
# 可以使用read_table，并指定分隔符
df1 = pd.read_table("./examples/ex1.csv", sep=',')
df1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# 有的文件不包含表头
df2 = pd.read_csv("./examples/ex2.csv", header=None)
df2

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# 也可以手动指定表头
df3 = pd.read_csv("./examples/ex2.csv", names=['a', 'b', 'c', 'd', 'message'])
df3

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
!cat./ examples / csv_mindex.csv

zsh:1: no such file or directory: cat./


In [7]:
# 当想要从多个列中形成一个分层索引，需要传入一个包含列序号或列名的列表
df4 = pd.read_csv("./examples/csv_mindex.csv", index_col=['key1', 'key2'])
df4

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [8]:
!cat./ examples / ex3.txt

zsh:1: no such file or directory: cat./


In [9]:
list(open("./examples/ex3.txt"))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [10]:
# 当分隔符由不同数量的空格分隔开时，可以使用正则表达式来处理
# \s+：至少匹配一个空格
result = pd.read_table('./examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [11]:
!cat./ examples / ex4.csv

zsh:1: no such file or directory: cat./


In [12]:
# 可以跳过指定的行
pd.read_csv('./examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
# 缺失值处理
!cat./ examples / ex5.csv

zsh:1: no such file or directory: cat./


In [14]:
result1 = pd.read_csv("./examples/ex5.csv", na_values=['NULL'])
result1

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [15]:
# 在字典中，每列可以指定不同缺失值标识
# 将message中的foo和something中的two置为NA
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [16]:
# 设置pandas设置显示行数
pd.options.display.max_rows = 10

In [17]:
# 只读取前5行
result = pd.read_csv("./examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [18]:
# 为了分块读入文件，可以指定chunksize作为每一块的行数
chunker = pd.read_csv("./examples/ex6.csv", chunksize=1000)
chunker

In [19]:
tot = pd.Series([], dtype='float64')
# 按照chunksize的大小来遍历目标文件，并对key列聚合获得技术值
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

In [20]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 将数据写入文件格式

In [21]:
data = pd.read_csv("./examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
data.to_csv("./output/out.csv")

In [23]:
# 也可以指定其他分隔符
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [24]:
# 将空值输出为NULL
data.to_csv(sys.stdout, na_rep='NULL', sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [25]:
# header和index不输出
data.to_csv(sys.stdout, sep="|", header=False, index=False)

one|1|2|3.0|4|
two|5|6||8|world
three|9|10|11.0|12|foo


In [26]:
# Series也可以写入csv
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv(sys.stdout, sep="|")

|0
2000-01-01|0
2000-01-02|1
2000-01-03|2
2000-01-04|3
2000-01-05|4
2000-01-06|5
2000-01-07|6


### 使用分隔格式

In [27]:
!cat./ examples / ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [28]:
# 对于任何带有单字符分隔符的文件，可以使用Python的内建csv模块。
# 要使用它，需要将任一打开的文件或文件型对象传给csv.reader
import csv

f = open("./examples/ex7.csv")
reader = csv.reader(f)
for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [29]:
with open("./examples/ex7.csv") as f:
    lines = list(csv.reader(f))

In [30]:
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [31]:
header, values = lines[0], lines[1:]

In [32]:
header

['a', 'b', 'c']

In [33]:
values

[['1', '2', '3'], ['1', '2', '3']]

In [34]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [35]:
# 字典转化为DataFrame
df = pd.DataFrame(data_dict)
df

,a,b,c
0,1,2,3
1,1,2,3


In [46]:
# 当CSV文件有不同的风格时，可以根据不同的分隔符、字符串引用约定或行终止的一种新的格式时，
# 可以使用csv.Dialect定义一个简单的子类
class my_dialect(csv.Dialect):
    lineterminator = '\n'  #行终止符
    delimiter = ';'  # 分隔符
    quotechar = '"'  # 用在特殊字符字段中的引号
    quoting = csv.QUOTE_MINIMAL  # 引用惯例，默认为csv.QUOTE_MINIMAL：是只使用特殊字符，如分隔符


f = open("./examples/ex7.csv")
reader = csv.reader(f, dialect=my_dialect)
for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [48]:
# 也可以不定义子类，直接使用分隔符关键字
reader = csv.reader(f, delimiter='|')
f.close()

In [49]:
# 写入csv文件
with open("./examples/mydata.csv", 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow((1, 2, 3))
    writer.writerow((4, 5, 6))
    writer.writerow((7, 8, 9))

### JSON数据